## Vehicle update order

Suppose we want to perform a discrete-event simulation with online (re)planning of crossing times upon new arrivals to the network. In that case, we need to be careful in which order we move the vehicles forward such that the follow constraints are satisfied at all times. This roughly means that the vehicle furthest on a route needs to be processed before all other vehicles on that route. All these types of precedence constraints give rise to a certain "vehicle update order", which can be directly computed from the disjunctive graph, when available, but we can also use the schedule times y and the routes in the network to implicitly reconstruct the disjunctive graph. We do not return any representation of the disjunctive graph at this point, so we compute the vehicle order from the crossing times. Once we have a suitable way to encode the disjunctive graph, we can use the following code to test the implementation.

In [1]:
def edge_order(G, routes):
    """Compute the order in which edges should be visited for processesing vehicles."""    
    # Generate list of tuples (order, v, w) where order is the number of edges of
    # the longest path to the last edge of some route.
    # This implementation just keeps taking maximum values along routes until
    # the path-lengths (number of edges) does not change anymore, so it can be
    # done more efficiently.
    edges = {}
    changed = True
    while changed:
        changed = False
        for route in routes:
            for order in range(len(route) - 1):
                i = len(route) - order - 1
                v, w = route[i-1], route[i]
    
                if order > 0: # >= (preceding edge order) + 1
                    order = max(order, edges[route[i], route[i+1]] + 1)
                if (v, w) in edges: # >= (order of edge for other route)
                    order = max(order, edges[v, w])
    
                if (v,w) not in edges or edges[v, w] != order:
                    changed = True
                edges[v, w] = order

    # sort by order
    return [edge for edge, order in sorted(edges.items(), key=lambda item: item[1])]

def vehicle_order(G, routes, positions):
    """Compute the order in which vehicles should be processed."""
    vehicles = []
    for edge in edge_order(G, routes):
        # get vehicles in this edge
        edge_vehicles = []
        for vehicle, position in positions.items():
            l, k = vehicle
            v, w, rel_pos = current_edge(G, routes[l], position)
            if (v, w) == edge:
                edge_vehicles.append((rel_pos, vehicle))
        
        # sort according to relative position and add to global list
        vehicles.extend([vehicle for rel_pos, vehicle in reversed(sorted(edge_vehicles, key=lambda x: x[0]))])

    return vehicles